In [ ]:
##　初回　必須　　0001-01

import numpy as np
import pandas as pd
import pickle
import io

In [ ]:
###　初回　必須　　0010-10
### アンロード
race_info = pd.read_pickle('data/race_info.pickle')
race_results = pd.read_pickle('data/race_results.pickle')
horse_info = pd.read_pickle('data/horse_info.pickle')
horse_peds = pd.read_pickle('data/horse_peds.pickle')
race_return = pd.read_pickle('data/race_return.pickle')

In [ ]:
###　初回　必須　　0011-10
###　------------------------------
### 対象のrace_resultsを抽出　－－＞　taisho_race　とする
###　------------------------------
###　----------分析用データの条件設定(ｔａｉｓｈｏ＿ｒａｃｅの処理)
taisho_race = race_results[(race_results['kaisai_nen'] == '2023') & (race_results['data_kubun'] == '7')]
#taisho_race = race_results[race_results['kaisai_nen'] == '2023']
taisho_race['ketto_toroku_bango'] = taisho_race['ketto_toroku_bango'].astype('int')
#＞＞＞## 対象のrace_resultsのキー情報抽出
#＞＞＞taisho_race = taisho_race.loc[:, ["ketto_toroku_bango", "race_nengappi"]]
###　レース年月日を日付タイプへ　かつ変数名を　date　へ
taisho_race["date"] = pd.to_datetime(taisho_race["race_nengappi"])
taisho_race.drop(["race_nengappi"],axis=1,inplace=True)



In [ ]:
### 不要
taisho_race = taisho_race.loc['202306050811']

In [ ]:
###　初回必須　　0012-10
###　----------horse_infoのインデックスを血統登録番号へ　　将来的には_１へ移す？
horse_info = horse_info.reset_index('race_id')
horse_info = horse_info.set_index('ketto_toroku_bango')


In [ ]:
###　初回必須　　0020-01
from tqdm.notebook import tqdm

class HorseResults:
    def __init__(self, horse_info_):
        self.horse_results = horse_info_[['race_nengappi','kakutei_chakujun','tansho_odds','kakutoku_honshokin','kohan_3f']]
        self.preprocessing()
        
    def preprocessing(self):
        df = self.horse_results.copy()
        df['date'] = pd.to_datetime(df["race_nengappi"],format='%Y%m%d')
        df.drop(["race_nengappi"],axis=1,inplace=True)
        self.horse_results = df

    def average(self, horse_id_list, date, n_samples='all'):
        target_df = self.horse_results.loc[horse_id_list]
#        target_df = target_df.set_index('ketto_toroku_bango') 

        #過去何走分取り出すか指定
        if n_samples == 'all':
            filterd_df = target_df[target_df['date'] < date]    
        elif n_samples > 0:
            filterd_df = target_df[target_df['date'] < date].sort_values\
                ('date',ascending=False).groupby(level=0).head(n_samples)
        else:
            raise Exception('n_samples must be >0')
        average = filterd_df.groupby(level=0)[['kakutei_chakujun','kakutoku_honshokin','kohan_3f']].mean()
        return average.rename(columns=\
                              {'kakutei_chakujun':'chakujun_{}R'.format(n_samples),\
                               'kakutoku_honshokin':'shokin_{}R'.format(n_samples),\
                               'kohan_3f':'kohan_3f_{}R'.format(n_samples)})
    
    def merge(self, results, date, n_samples='all'):
        df = results[results['date'] == date]
        df.reset_index(inplace=True)
        horse_id_list = df['ketto_toroku_bango']
        merged_df = df.merge(self.average(horse_id_list, date, n_samples),\
                             left_on='ketto_toroku_bango',right_index=True,how='left')
        return merged_df
    
    def merge_all(self, results, n_samples='all'):
        date_list = results['date'].unique()
        merged_df = pd.concat([self.merge(results, date, n_samples) for date in tqdm(date_list)])
        return merged_df

In [ ]:
hr = HorseResults(horse_info)
results_3R = hr.merge_all(taisho_race,n_samples=3)
results_3R.set_index('race_id', inplace=True)
results_5R = hr.merge_all(results_3R,n_samples=5)
results_5R.set_index('race_id', inplace=True)
results_9R = hr.merge_all(results_5R,n_samples=9)
results_9R.set_index('race_id', inplace=True)
results_9R.head()

In [ ]:
###　オブジェクトをいったん変数へ
#model_input = hr.merge_all(taisho_race)
model_input = results_9R
###　必要に応じて保存　日付変更
model_input.to_pickle('data/model_input_20240406.pickle')

In [18]:
###　再開時、ここからロード
model_input = pd.read_pickle('data/model_input_20240406.pickle')

In [19]:
### 型変換　基本ｉｎｔ
#model_input['record_id'] = model_input['record_id'].astype(int)
model_input['keibajo_code'] = model_input['keibajo_code'].astype('category')
model_input['kaisai_kai'] = model_input['kaisai_kai'].astype(int)
model_input['kaisai_nichime'] = model_input['kaisai_nichime'].astype(int)
model_input['race_bango'] = model_input['race_bango'].astype(int)
model_input['wakuban'] = model_input['wakuban'].astype(int)
model_input['umaban'] = model_input['umaban'].astype(int)
model_input['ketto_toroku_bango'] = model_input['ketto_toroku_bango'].astype('category')
#model_input['bamei'] = model_input['bamei'].astype(int)
model_input['umakigo_code'] = model_input['umakigo_code'].astype('category')
model_input['seibetsu_code'] = model_input['seibetsu_code'].astype('category')
model_input['hinshu_code'] = model_input['hinshu_code'].astype('category')
model_input['barei'] = model_input['barei'].astype(int)
model_input['tozai_shozoku_code'] = model_input['tozai_shozoku_code'].astype('category')
model_input['chokyoshi_code'] = model_input['chokyoshi_code'].astype('category')
model_input['banushi_code'] = model_input['banushi_code'].astype('category')
model_input['futan_juryo'] = model_input['futan_juryo'].astype(int)
model_input['kishu_code'] = model_input['kishu_code'].astype('category')
model_input['bataiju'] = model_input['bataiju'].astype(int)
#model_input['zogen_fugo'] = model_input['zogen_fugo'].astype(int)
#model_input['zogen_sa'] = model_input['zogen_sa'].astype(int)
model_input['kakutei_chakujun'] = model_input['kakutei_chakujun'].astype(int)
model_input['soha_time'] = model_input['soha_time'].astype(int)
#model_input['chakusa_code_1'] = model_input['chakusa_code_1'].astype(int)
model_input['corner_1'] = model_input['corner_1'].astype(int)
model_input['corner_2'] = model_input['corner_2'].astype(int)
model_input['corner_3'] = model_input['corner_3'].astype(int)
model_input['corner_4'] = model_input['corner_4'].astype(int)
model_input['tansho_odds'] = model_input['tansho_odds'].astype(int)
model_input['tansho_ninkijun'] = model_input['tansho_ninkijun'].astype(int)
model_input['kakutoku_honshokin'] = model_input['kakutoku_honshokin'].astype(int)
#model_input['kakutoku_fukashokin'] = model_input['kakutoku_fukashokin'].astype(int)
model_input['kohan_4f'] = model_input['kohan_4f'].astype(int)
model_input['kohan_3f'] = model_input['kohan_3f'].astype(int)
#model_input['time_sa'] = model_input['time_sa'].astype(int)
#model_input['mining_kubun'] = model_input['mining_kubun'].astype('category')
#model_input['yoso_soha_time'] = model_input['yoso_soha_time'].astype(int)
#model_input['yoso_gosa_plus'] = model_input['yoso_gosa_plus'].astype('category')
#model_input['yoso_gosa_minus'] = model_input['yoso_gosa_minus'].astype('category')
#model_input['yoso_juni'] = model_input['yoso_juni'].astype(int)
#model_input['kyakushitsu_hantei'] = model_input['kyakushitsu_hantei'].astype('category')


In [20]:
###　目的変数の設定　4着以下はすべて４
clip_rank = lambda x: x if x<4 else 4
model_input['rank'] = model_input['kakutei_chakujun'].map(clip_rank)

###　着順０は除外
model_input = model_input[model_input['kakutei_chakujun'] > 0]

In [21]:
###　説明変数候補以外はドロップ
model_input.drop(['record_id',\
                  'data_kubun',\
                  'data_sakusei_nengappi',\
                  'kaisai_nen',\
                  'kaisai_tsukihi',\
                  'kaisai_nichime',\
                  'race_bango',\
                  'bamei',\
                  'zogen_fugo',\
                  'zogen_sa',\
                  'soha_time',\
                  'chakusa_code_1',\
                  'date',\
                  'corner_1',\
                  'corner_2',\
                  'corner_3',\
                  'corner_4',\
                  'ketto_toroku_bango',\
                  'umakigo_code',\
                  'hinshu_code',\
                  'kakutei_chakujun',\
                  'tansho_odds',\
                  'tansho_ninkijun',\
                  'kakutoku_honshokin',\
                  'kakutoku_fukashokin',\
                  'kohan_4f',\
                  'kohan_3f',\
                  'time_sa',\
                  'mining_kubun',\
                  'yoso_soha_time',\
                  'yoso_gosa_plus',\
                  'yoso_gosa_minus',\
                  'yoso_juni',\
                  'kyakushitsu_hantei'\
                  ],axis=1,inplace=True)

In [22]:
## カテゴリ変数をＤＵＭＭＹ化
model_input_d = pd.get_dummies(model_input)

In [23]:
###　初期値を０埋め
model_input_d.fillna(0,inplace=True)

In [24]:
###　モデル実施　データをトレーニング、テストにわける
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = model_input_d.drop(['rank'],axis=1)
y = model_input_d['rank']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)

In [27]:
model_input_d['rank'].value_counts()

rank
0    36898
1    10376
Name: count, dtype: int64

In [26]:
clip_rank = lambda x: 1 if x<4 else 0
model_input_d['rank'] = model_input_d['rank'].map(clip_rank)

In [28]:
#　サンプル数を変更　4着以下を　１，２，３着と同数にする

from imblearn.under_sampling import RandomUnderSampler

rank_1 = y_train.value_counts()[1]
rank_2 = y_train.value_counts()[2]
rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(sampling_strategy={1:rank_1,2:rank_2,3:rank_3,4:rank_1},random_state=71)
X_train_rus,y_train_rus = rus.fit_resample(X_train,y_train)

In [ ]:
clip_rank = lambda x: 1 if x<4 else 0
model_input_d['rank'] = model_input_d['rank'].map(clip_rank)

In [ ]:
model_input_d['rank'].head()

In [ ]:
pd.Series(y_train_rus).value_counts()

In [ ]:
###　モデル実施　データをトレーニング、テストにわける
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = model_input_d.drop(['rank'],axis=1)
y = model_input_d['rank']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)

In [ ]:
model = LogisticRegression()
model.fit(X_train_rus,y_train_rus)

print(model.score(X_train,y_train),model.score(X_test,y_test))

In [ ]:
#date_listy_pred = model.predict(X_test)
y_pred = model.predict_proba(X_test)

In [ ]:
y_pred

In [ ]:
pred_df = pd.DataFrame({'pred':y_pred,'actual':y_test})

In [ ]:
pred_df[pred_df['pred']==1]['actual'].value_counts()

In [ ]:
coefs = pd.Series(model.coef_[0],index=X.columns).sort_values()

In [ ]:
coefs

In [ ]:
#model_input[model_input['race_id'] == '2023010506010101'].to_csv('model_input_sel.csv')
model_input.loc['2023010506010101'].to_csv('model_input_sel.csv')


In [ ]:
###　？？？
sample_date = taisho_race['date'][0]

##
## date taisho_race の　date　を　日付で　ソート　＆　重複けし
## 日付がなくなるまで　ＦＯＲで繰り返す
sample_date

In [ ]:
horse_id_list = taisho_race[taisho_race['date'] == sample_date]['ketto_toroku_bango'].unique()

In [ ]:
#日付取得
sample_date = taisho_race['date'][0]
#対象馬リスト　その日走った馬
horse_id_list = taisho_race[taisho_race['date'] == sample_date]['ketto_toroku_bango'].unique()
#　LOCがうまくいかないのでＬＩＳＴ２作成
horse_id_list2 = pd.DataFrame(horse_id_list,columns=['ketto_toroku_bango'])
#type(horse_id_list2)
horse_id_list2['ketto_toroku_bango'] = horse_id_list2['ketto_toroku_bango'].astype('int')

#　候補ｒｅｓｕｌｔｓ　を自力で作成
kouho_results = pd.merge(horse_id_list2,hr.horse_results,left_on='ketto_toroku_bango',right_on='ketto_toroku_bango',how='left')
#hr.horse_results[(hr.horse_results.index == '2016106582')]
kouho_results = kouho_results.set_index('ketto_toroku_bango')

In [ ]:
#hr.horse_results.loc[horse_id_list2]
hr.preprocessing()

In [ ]:
target_df = kouho_results
target_df[target_df['date'] < sample_date].groupby(level=0)[['kakutei_chakujun','kakutoku_honshokin','kohan_3f']].mean()

In [ ]:
#horse_info['race_nengappi'].value_counts()
#horse_info['date'] = pd.to_datetime(horse_info["race_nengappi"])
#horse_info.isnull().sum()
horse_info['race_nengappi']
taisho_race

In [ ]:
horse_info = pd.read_pickle('horse_info.pickle')
#　馬情報テーブルにレース年月日追加
horse_info = horse_info[horse_info['data_kubun'] == '7']
#　馬情報項目セレクト
race_data_sel = horse_info[['ketto_toroku_bango',\
                           'kakutei_chakujun',\
                           'corner_1',\
                           'corner_4',\
                           'tansho_odds',\
                            'tansho_ninkijun',\
                           'kakutoku_honshokin',\
                            'kohan_3f',\
                            'time_sa',\
                            'yoso_juni',\
                            'kyakushitsu_hantei',\
                            'race_nengappi']]
race_data_sel['ketto_toroku_bango'] = race_data_sel['ketto_toroku_bango'].astype('int')
race_data_sel['kakutei_chakujun'] = race_data_sel['kakutei_chakujun'].astype('int')
race_data_sel['corner_1'] = race_data_sel['corner_1'].astype('int')
race_data_sel['corner_4'] = race_data_sel['corner_4'].astype('int')
race_data_sel['tansho_odds'] = race_data_sel['tansho_odds'].astype('int')
race_data_sel['tansho_ninkijun'] = race_data_sel['tansho_ninkijun'].astype('int')
race_data_sel['kakutoku_honshokin'] = race_data_sel['kakutoku_honshokin'].astype('int')
race_data_sel['time_sa'] = race_data_sel['time_sa'].astype('int')
race_data_sel['kohan_3f'] = race_data_sel['kohan_3f'].astype('int')
race_data_sel['yoso_juni'] = race_data_sel['yoso_juni'].astype('int')
race_data_sel['kyakushitsu_hantei'] = race_data_sel['kyakushitsu_hantei'].astype('int')
race_data_sel['race_nengappi'] = race_data_sel['race_nengappi'].astype('int')
race_data_sel.isnull().sum()
horse_info['tansho_odds'].value_counts

In [ ]:
race_data_sel['tansho_odds'].value_counts()


In [ ]:
horse_info = pd.read_csv('horse_info.csv')
horse_info['tansho_odds'].value_counts()

In [ ]:
### key_info 
taisho_race = race_data[race_data['kaisai_nen'] == '2023']
taisho_race = taisho_race.loc[:, ["ketto_toroku_bango", "race_nengappi"]]
taisho_race.head